# Importing Libraries

In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt

# Image Resizing Function

In [2]:
def resize_image(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]
    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))
    resized = cv2.resize(image, dim, interpolation=inter)
    return resized

# Face Feature Extraction Function

In [3]:
def extract_face_features(folder_path, face_cascade, target_size=(100, 100)):
    features = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        resized_img = resize_image(img, width=800)
        gray = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
        for (x, y, w, h) in faces:
            face_roi = gray[y:y+h, x:x+w]
            face_roi_resized = cv2.resize(face_roi, target_size)
            features.append(face_roi_resized)
    return features

# Data Preprocessing Function

In [4]:
def preprocess_data(faces, label):
    X = []
    y = []
    for face in faces:
        face = cv2.resize(face, (100, 100))
        X.append(face.flatten())
        y.append(label)
    return X, y


# Processing Test Image Function

In [6]:
def process_test_image(test_image_path, face_cascade, svm_model):
    test_img = cv2.imread(test_image_path)
    test_gray = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(test_gray, scaleFactor=1.1, minNeighbors=5)
    test_faces = []
    for (x, y, w, h) in faces:
        face_roi = test_gray[y:y+h, x:x+w]
        face_roi_resized = cv2.resize(face_roi, (100, 100))
        test_faces.append(face_roi_resized)
    X_test = np.array([face.flatten() for face in test_faces])
    y_pred_test = svm_model.predict(X_test)
    for (x, y, w, h), gender in zip(faces, y_pred_test):
        gender_label = 'Female' if gender == 0 else 'Male'
        cv2.rectangle(test_img, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(test_img, gender_label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    test_img_resized = cv2.resize(test_img, (400, 400))
    cv2.imshow('Test Image', test_img_resized)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

# Dataset Paths

In [7]:
folder_path1 = r'C:\Users\Hello\Downloads\Male and Female face dataset\Female Faces'
folder_path2 = r'C:\Users\Hello\Downloads\Male and Female face dataset\Male Faces'

# Cascade Classifier Initialization

In [8]:
cascade_path = os.path.join(cv2.data.haarcascades, 'haarcascade_frontalface_default.xml')
face_cascade = cv2.CascadeClassifier(cascade_path)

# Extracting Face Features

In [9]:
female_faces = extract_face_features(folder_path1, face_cascade)
male_faces = extract_face_features(folder_path2, face_cascade)

# Preprocessing Data

In [10]:
X_female, y_female = preprocess_data(female_faces, 0)
X_male, y_male = preprocess_data(male_faces, 1)
X = np.array(X_male + X_female)
y = np.array(y_male + y_female)


# Train-Test Split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# SVM Model Training

In [12]:
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)


SVC(kernel='linear')

# Process Test Image Using SVM Model

In [27]:
test_image_path = r'C:\Users\Hello\Desktop\Test images\y.jpg'
process_test_image(test_image_path, face_cascade, svm_model)

# Close the window after verifying Gender of the test image. And run the next cell.

# Live Gender Detection Using SVM Model

In [14]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_roi = gray[y:y+h, x:x+w]
        face_roi_resized = cv2.resize(face_roi, (100, 100))
        face_flattened = face_roi_resized.flatten()
        gender = svm_model.predict([face_flattened])
        gender_label = 'Female' if gender == 0 else 'Male'
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, gender_label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    cv2.imshow('Gender Detection - SVM', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# press small q to quit live detection.

# Evaluation Metrics for SVM Model

In [15]:
y_pred_svm = svm_model.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)

print("SVM Model:")
print("Accuracy:", accuracy_svm)
print("Precision:", precision_svm)
print("Recall:", recall_svm)
print("F1-score:", f1_svm)

SVM Model:
Accuracy: 0.9694719471947195
Precision: 0.9798761609907121
Recall: 0.9634703196347032
F1-score: 0.9716039907904834


# Random Forest Model Training

In [16]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

# Processing Test Image with Random Forest Model

In [28]:
test_image_path = r'C:\Users\Hello\Desktop\Test images\y.jpg'
process_test_image(test_image_path, face_cascade, rf_model)

# Live Gender Detection Using Random Forest

In [18]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_roi = gray[y:y+h, x:x+w]
        face_roi_resized = cv2.resize(face_roi, (100, 100))
        face_flattened = face_roi_resized.flatten()
        gender_rf = rf_model.predict([face_flattened])
        gender_label_rf = 'Female' if gender_rf == 0 else 'Male'
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, f'RF: {gender_label_rf}', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    cv2.imshow('Gender Detection - Random Forest', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# Random Forest Model Evaluation

In [19]:
y_pred_rf = rf_model.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)


print("\nRandom Forest Model:")
print("Accuracy:", accuracy_rf)
print("Precision:", precision_rf)
print("Recall:", recall_rf)
print("F1-score:", f1_rf)


Random Forest Model:
Accuracy: 0.9752475247524752
Precision: 0.9644444444444444
Recall: 0.9908675799086758
F1-score: 0.9774774774774775


# KNN Model Hyperparameter Tuning

In [20]:
param_grid = {'n_neighbors': range(1, 21)}
knn_grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='accuracy')
knn_grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 21)}, scoring='accuracy')

In [21]:
best_k = knn_grid.best_params_['n_neighbors']
print(best_k)

1


# KNN Model Training

In [22]:
knn_model = KNeighborsClassifier(n_neighbors=best_k)
knn_model.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=1)

# Processing Test Image Using KNN Model

In [29]:
test_image_path = r'C:\Users\Hello\Desktop\Test images\y.jpg'
process_test_image(test_image_path, face_cascade, knn_model) 

# Live Gender Detection Using KNN

In [24]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_roi = gray[y:y+h, x:x+w]
        face_roi_resized = cv2.resize(face_roi, (100, 100))
        face_flattened = face_roi_resized.flatten()
        gender_knn = knn_model.predict([face_flattened])
        gender_label_knn = 'Female' if gender_knn == 0 else 'Male'
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, f'KNN: {gender_label_knn}', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    cv2.imshow('Gender Detection - KNN', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# KNN Model Evaluation

In [25]:
y_pred_knn = knn_model.predict(X_test)
accuracy_knn = accuracy_score(y_test, y_pred_knn)
precision_knn = precision_score(y_test, y_pred_knn)
recall_knn = recall_score(y_test, y_pred_knn)
f1_knn = f1_score(y_test, y_pred_knn)

print("\nKNN Model:")
print("Accuracy:", accuracy_knn)
print("Precision:", precision_knn)
print("Recall:", recall_knn)
print("F1-score:", f1_knn)



KNN Model:
Accuracy: 0.9760726072607261
Precision: 0.9686567164179104
Recall: 0.9878234398782344
F1-score: 0.9781461944235116
